In [12]:
from mip import Model, xsum, minimize, BINARY, Constr

qmin = [1/2, 1/2]
qmax = [1, 3/4]

wmin = [1/2, 3/4]
wmax = [1, 7/8]

ncomps = len(qmin)
I = range(ncomps)

alpha = [(wmax[i] - wmin[i])/(qmax[i] - qmin[i]) for i in I]
beta = [wmin[i] - qmin[i] * alpha[i] for i in I]

Q = 0

m = Model()

theta = [m.add_var(var_type=BINARY) for _ in I]
q = [m.add_var() for _ in I]

m.objective = minimize(xsum(alpha[i] * q[i] + beta[i] - beta[i] * (1 - theta[i]) for i in I))

#Constraints:
for i in I:
    m += q[i] <= qmax[i] * theta[i]
    m += q[i] >= qmin[i]* theta[i]
m += xsum(q[i] for i in I) == Q, "asdfsdf"
print(m.constr_by_name('Qtot'))
m += xsum(q[i] for i in I) == 0.75, "Qtot"

m.optimize()


[bool(theta[i].x) for i in I], [q[i].x for i in I]

None
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 1 substitutions
Cgl0004I processed model has 4 rows, 2 columns (1 integer (1 of which binary)) and 8 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 10.000%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 0.75
Cbc0038I Relaxing continuous gives 0.75
Cbc0038I Before mini branch and bound, 1 integers at bound fixed and 1 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of 0.75 - took 0.00 seconds
Cbc0012I Integer solution of 0.75 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 0.75, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Total time (CPU seconds):       0.00   (Wallclock seco

([False, False], [None, None])

In [1]:
from cosmiclib import Compressor, Optimizer

In [2]:
my_compressor = Compressor('my compressor', qmin=0.5, qmax=1, wmin=0.5, wmax=1)

In [3]:
my_optimizer = Optimizer(compressors=[my_compressor])

In [4]:
my_optimizer.setup_problem()

In [5]:
my_optimizer.beta

[0.0]

In [10]:
my_optimizer.find_opt(100)

Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Starting solution of the Linear programming relaxation problem using Primal Simplex



([False], [None])